# Analyzing Profitable App Profiles for App Store & Google Play Markets

Our aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets. <br> 

<b>Scenario:</b> <br>
We're working as data analysts for a company that builds Android and iOS mobile apps, and our job is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

Opening and Exploring the Data
As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

<img src="py1m8_statista.png" width = "600" height = "300">

Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our purpose:

A data set containing data about approximately ten thousand Android apps from Google Play. You can download the data set directly from this link.
A data set containing data about approximately seven thousand iOS apps from the App Store. You can download the data set directly from this link.
Let's start by opening the two data sets and then continue with exploring the data.

Data used in this project: <br>
Approx 10,000 apps collected from Google Play Store (as of Aug 2018) <br>
Approx 7,000 apps collected from Apple Store (as of July 2017)

# Importing Dataset

In [91]:
import pandas as pd
ios = pd.read_csv("AppleStore.csv")
android = pd.read_csv("googleplaystore.csv")

ios_header = list(ios.columns) #get header of data from dataframe
ios_data = ios.to_numpy().tolist() #convert dataframe data to list of lists

android_header = list(android.columns) #get header of data from dataframe
android_data = android.to_numpy().tolist() #convert dataframe data to list of lists


In [92]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [93]:
#explore_data(ios_data,1,10,rows_and_columns = True)
android.info()
ios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
App               10841 non-null object
Category          10841 non-null object
Rating            9367 non-null float64
Reviews           10841 non-null object
Size              10841 non-null object
Installs          10841 non-null object
Type              10840 non-null object
Price             10841 non-null object
Content Rating    10840 non-null object
Genres            10841 non-null object
Last Updated      10841 non-null object
Current Ver       10833 non-null object
Android Ver       10838 non-null object
dtypes: float64(1), object(12)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7197 entries, 0 to 7196
Data columns (total 16 columns):
id                  7197 non-null int64
track_name          7197 non-null object
size_bytes          7197 non-null int64
currency            7197 non-null object
price               7197 non-null float64
rating_

We see that the Google Play data set has 10841 apps and 13 columns. At a quick glance, the columns that might be useful for the purpose of our analysis are 'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price', and 'Genres'.

We have 7197 iOS apps in this data set, and the columns that may be usefulare: 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', and 'prime_genre'.

# Delete Wrong Data

In [94]:
#print(android.isnull().values.any()) #checks if dataframe has any NULL values, works only on dataframe and not list
#print(ios.isnull().values.any()) #checks if dataframe has any NULL values, works only on dataframe and not list

#null_columns = android.columns[android.isnull().any()]
#print(null_columns)

#print(android[android.isnull().any(axis=1)][null_columns]) #prints rows with all the null values
#print(android[android.isnull().any(axis=1)][null_columns]["Android Ver"].to_string())

incorrect_rating = [android[android["Rating"] > 5]]
for row in incorrect_rating:
    print(row.index) # find row(s) where rating value is more than 5 

print(android_data[10472])
print("Before removing wrong data: {}".format(len(android_data))
del(android_data[10472])
print("After removing wrong data: {}".format(len(android_data))

Int64Index([10472], dtype='int64')
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', 19.0, '3.0M', '1,000+', 'Free', '0', 'Everyone', nan, 'February 11, 2018', '1.0.19', '4.0 and up', nan]
10841
10840


# Remove Duplicate App Data

In [95]:
#remove duplicated app names from android data
unique_name = []
duplicated_name = []

for app in android_data:
    app_name = app[0]
    if app_name in unique_name:
        duplicated_name.append(app_name)
    else:
        unique_name.append(app_name)
print("Android dataset contains a total of {} duplicated entries by name."
      .format(len(duplicated_name)))

Android dataset contains a total of 1181 duplicated entries by name.


In [96]:
#examine the first app in duplicated apps list
for app in android_data:
    if app[0] == duplicated_name[0]:
        print(app)

print(android_header)

['Quick PDF Scanner + OCR FREE', 'BUSINESS', 4.2, '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', 4.2, '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', 4.2, '80804', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


To get the latest version of the duplicated app -> instead of removing a random entry, we will keep the app which has the highest number of reviews and remove the rest.

We found a total of 1181 rows of duplicated app names while there was a total of 10840 rows (excluding header).
After removing duplicates, there should be 9659 rows.

In [97]:
expected_length = len(android_data) - 1181
print(expected_length)

9659


In [98]:
reviews_max = {}
for app in android_data:
    app_name = app[0]
    n_reviews = float(app[3])
    if app_name in reviews_max and reviews_max[app_name] < n_reviews:
        reviews_max[app_name] = n_reviews
    elif app_name not in reviews_max:
            reviews_max[app_name] = n_reviews
print(len(reviews_max))

9659


In [99]:
android_clean = []
already_added = []

for app in android_data:
    app_name = app[0]
    n_reviews = float(app[3])
    if (reviews_max[app_name] == n_reviews) and (app_name not in already_added):
        android_clean.append(app)
        already_added.append(app_name)

print(len(android_clean))
explore_data(android_clean, 0, 5, True)

9659
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', 4.1, '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', 4.7, '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']
['Sketch - Draw & Paint', 'ART_AND_DESIGN', 4.5, '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']
['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', 4.3, '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']
['Paper flowers instructions', 'ART_AND_DESIGN', 4.4, '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']
Number of rows: 9659
Number of columns: 13


In [100]:
#remove duplicated app names from ios data
unique_name = []
duplicated_name = []

for app in ios_data:
    app_name = app[0]
    if app_name in unique_name:
        duplicated_name.append(app_name)
    else:
        unique_name.append(app_name)
print("IOS dataset contains a total of {} duplicated entries by name."
      .format(len(duplicated_name)))    

IOS dataset contains a total of 0 duplicated entries by name.


# Removing Non-English characters

Now we will remove all the apps which contain non-English alphabets. <br>
The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII (American Standard Code for Information Interchange) system. Based on this number range, we can build a function that detects whether a character belongs to the set of common English characters or not. If the number is equal to or less than 127, then the character belongs to the set of common English characters.

In [101]:
def check_english(string):
    count_non_ascii = 0
    for character in string:
        if ord(character) > 127:
            count_non_ascii += 1
    if(count_non_ascii > 3):
        return False       
    else: return True

In [102]:
check_english("PPS -《欢欢")
#check_english("Instagram")

True

If we're going to use the function we've created, we'll lose useful data since many English apps will be incorrectly labeled as non-English. To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with up to three emoji or other special characters will still be labeled as English. Our filter function is still not perfect, but it should be fairly effective.

In [103]:
android_english = []
for app in android_clean:
    app_name = app[0]
    #check if app_name meets the criteria of the check_english function
    if check_english(app_name):
        android_english.append(app)
print(len(android_english))

#get all the columns which contain rating values in the IOS dataset
column_name = [col for col in ios.columns if "rating" in col]

incorrect_rating = ios[ios["user_rating"] > 5]
print(incorrect_rating.index) #no errors in the ratings 

ios_english = []
print(len(ios_data))
for app in ios_data:
    app_name = app[1]
    if check_english(app_name):
        ios_english.append(app)
print(len(ios_english))

9614
Int64Index([], dtype='int64')
7197
6183


After removing apps that contain more than 2 non-english characters: <br>
we have 6183 rows in the IOS dataset, down from 7197 <br>
and 9614 rows in the Google Play Store dataset, down from 9659 <br>
<br>
So far, we have removed:
1. Inaccurate Data
2. Duplicated App Entries
3. Non-English Apps

# Separating Free Apps from Paid Apps

In [104]:
#ios
#print(len(ios_english))
ios_free_apps = []
ios_paid_apps = []
for app in ios_english:
    price = app[4]
    if price == 0:
        ios_free_apps.append(app)
    else: ios_paid_apps.append(app)

#android
#print(len(android_english))
android_free_apps = []
android_paid_apps = []
for app in android_english:
    price = float(app[7].replace("$",""))
    if price == 0:
        android_free_apps.append(app)
    else: android_paid_apps.append(app)

#android.info()
#ios.info()
print("There are a total of {} free ios apps and {} paid ios apps".format(len(ios_free_apps),len(ios_paid_apps)))
print("There are a total of {} free android apps and {} paid android apps".format(len(android_free_apps),len(android_paid_apps)))

There are a total of 3222 free ios apps and 2961 paid ios apps
There are a total of 8864 free android apps and 750 paid android apps


As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:
1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of what are the most common genres for each market. For this, we'll need to build frequency tables for a few columns in our data sets.

# Analyzing Free Apps

In [105]:
#create a frequency table
def freq_table(dataset, column_index):
    table = {}
    num_of_apps = 0
    for app in dataset:
        num_of_apps += 1
        value = app[column_index]
        if value in table:
            table[value] += 1
        else: table[value] = 1
    #print(table)
    
    table_percentage = {}
    for key in table:
        percentage = (table[key]/num_of_apps) * 100
        table_percentage[key] = round(percentage,5)
    #print(table_percentage)
    return table_percentage

    
def display_table(dataset, column_index):
    table = freq_table(dataset, column_index)
    for key, value in sorted(table.items(), key=lambda item: item[1], reverse = True):
        print("{}: {}".format(key, value))
        
print("Free ios app by genres (percentage) \n")       
display_table(ios_free_apps, 11)


Free ios app by genres (percentage) 

Games: 58.16263
Entertainment: 7.8833
Photo & Video: 4.96586
Education: 3.66232
Social Networking: 3.28988
Shopping: 2.60708
Utilities: 2.51397
Sports: 2.14153
Music: 2.04842
Health & Fitness: 2.01738
Productivity: 1.73805
Lifestyle: 1.58287
News: 1.33457
Travel: 1.24146
Finance: 1.11732
Weather: 0.86903
Food & Drink: 0.80695
Reference: 0.55866
Business: 0.52762
Book: 0.43451
Navigation: 0.18622
Medical: 0.18622
Catalogs: 0.12415


We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Let's continue by examining the Genres and Category columns of the Google Play data set (two columns which seem to be related).

In [106]:
display_table(android_free_apps, 1)

FAMILY: 18.90794
GAME: 9.72473
TOOLS: 8.46119
BUSINESS: 4.59161
LIFESTYLE: 3.90343
PRODUCTIVITY: 3.89215
FINANCE: 3.70036
MEDICAL: 3.53114
SPORTS: 3.39576
PERSONALIZATION: 3.31679
COMMUNICATION: 3.23782
HEALTH_AND_FITNESS: 3.07987
PHOTOGRAPHY: 2.94449
NEWS_AND_MAGAZINES: 2.79783
SOCIAL: 2.66245
TRAVEL_AND_LOCAL: 2.33529
SHOPPING: 2.24504
BOOKS_AND_REFERENCE: 2.1435
DATING: 1.86146
VIDEO_PLAYERS: 1.79377
MAPS_AND_NAVIGATION: 1.39892
FOOD_AND_DRINK: 1.24097
EDUCATION: 1.162
ENTERTAINMENT: 0.95894
LIBRARIES_AND_DEMO: 0.93637
AUTO_AND_VEHICLES: 0.92509
HOUSE_AND_HOME: 0.82356
WEATHER: 0.80099
EVENTS: 0.71074
PARENTING: 0.65433
ART_AND_DESIGN: 0.64305
COMICS: 0.62049
BEAUTY: 0.59792


In [107]:
display_table(android_free_apps, -4)

Tools: 8.44991
Entertainment: 6.06949
Education: 5.34747
Business: 4.59161
Lifestyle: 3.89215
Productivity: 3.89215
Finance: 3.70036
Medical: 3.53114
Sports: 3.46345
Personalization: 3.31679
Communication: 3.23782
Action: 3.10244
Health & Fitness: 3.07987
Photography: 2.94449
News & Magazines: 2.79783
Social: 2.66245
Travel & Local: 2.32401
Shopping: 2.24504
Books & Reference: 2.1435
Simulation: 2.04197
Dating: 1.86146
Arcade: 1.85018
Video Players & Editors: 1.77121
Casual: 1.75993
Maps & Navigation: 1.39892
Food & Drink: 1.24097
Puzzle: 1.12816
Racing: 0.99278
Libraries & Demo: 0.93637
Role Playing: 0.93637
Auto & Vehicles: 0.92509
Strategy: 0.91381
House & Home: 0.82356
Weather: 0.80099
Events: 0.71074
Adventure: 0.6769
Comics: 0.60921
Art & Design: 0.59792
Beauty: 0.59792
Parenting: 0.49639
Card: 0.45126
Casino: 0.4287
Trivia: 0.41742
Educational;Education: 0.39486
Board: 0.38357
Educational: 0.37229
Education;Education: 0.33845
Word: 0.25948
Casual;Pretend Play: 0.23691
Music: 0.2

In [108]:
for app in android_free_apps:
    print(app[1], ":", [app[-4]])

ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design;Creativity']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design;Creativity']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design;Creativity']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN : ['Art & Design']
ART_AND_DESIGN 

COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
COMMUNICATION 

EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
EVENTS : ['Events']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE : ['Finance']
FINANCE 

LIBRARIES_AND_DEMO : ['Libraries & Demo']
LIBRARIES_AND_DEMO : ['Libraries & Demo']
LIBRARIES_AND_DEMO : ['Libraries & Demo']
LIBRARIES_AND_DEMO : ['Libraries & Demo']
LIBRARIES_AND_DEMO : ['Libraries & Demo']
LIBRARIES_AND_DEMO : ['Libraries & Demo']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle;Pretend Play']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTYLE : ['Lifestyle']
LIFESTY

MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : ['Medical']
MEDICAL : 

...

BUSINESS : ['Business']
BUSINESS : ['Business']
BUSINESS : ['Business']
BUSINESS : ['Business']
FAMILY : ['Entertainment']
FINANCE : ['Finance']
PRODUCTIVITY : ['Productivity']
TRAVEL_AND_LOCAL : ['Travel & Local']
BUSINESS : ['Business']
TOOLS : ['Tools']
EVENTS : ['Events']
SOCIAL : ['Social']
FINANCE : ['Finance']
TOOLS : ['Tools']
TOOLS : ['Tools']
BOOKS_AND_REFERENCE : ['Books & Reference']
FINANCE : ['Finance']
BOOKS_AND_REFERENCE : ['Books & Reference']
BOOKS_AND_REFERENCE : ['Books & Reference']
COMMUNICATION : ['Communication']
LIFESTYLE : ['Lifestyle']
FAMILY : ['Simulation']
FAMILY : ['Simulation;Pretend Play']
BOOKS_AND_REFERENCE : ['Books & Reference']
FAMILY : ['Casual']
FAMILY : ['Educational']
GAME : ['Racing']
GAME : ['Racing']
BOOKS_AND_REFERENCE : ['Books & Reference']
FAMILY : ['Simulation']
SOCIAL : ['Social']
LIFESTYLE : ['Lifestyle']
GAME : ['Racing']
FAMILY : ['Education;Action & Adventure']
FAMILY : ['Simulation']
FAMILY : ['Simulation']
FAMILY : ['Entertainmen

FAMILY : ['Entertainment']
TOOLS : ['Tools']
GAME : ['Adventure']
GAME : ['Arcade']
GAME : ['Adventure']
FAMILY : ['Casual']
FAMILY : ['Role Playing']
GAME : ['Action']
FAMILY : ['Casual;Brain Games']
GAME : ['Adventure']
SHOPPING : ['Shopping']
VIDEO_PLAYERS : ['Video Players & Editors']
FAMILY : ['Simulation']
FAMILY : ['Educational']
FAMILY : ['Casual']
FAMILY : ['Simulation']
FAMILY : ['Educational']
FAMILY : ['Simulation']
FAMILY : ['Simulation']
FAMILY : ['Casual']
LIFESTYLE : ['Lifestyle']
FAMILY : ['Entertainment']
FAMILY : ['Casual;Pretend Play']
FAMILY : ['Casual']
FAMILY : ['Educational']
FAMILY : ['Educational']
MEDICAL : ['Medical']
FAMILY : ['Casual']
FAMILY : ['Educational']
FAMILY : ['Casual']
FAMILY : ['Educational']
FAMILY : ['Educational']
FAMILY : ['Educational']
FAMILY : ['Role Playing']
FAMILY : ['Casual']
FAMILY : ['Casual']
FAMILY : ['Casual']
FAMILY : ['Role Playing']
FAMILY : ['Role Playing']
FAMILY : ['Role Playing']
FAMILY : ['Educational']
FAMILY : ['Simula

PERSONALIZATION : ['Personalization']
BUSINESS : ['Business']
SOCIAL : ['Social']
FAMILY : ['Role Playing']
LIFESTYLE : ['Lifestyle']
GAME : ['Board']
PERSONALIZATION : ['Personalization']
BUSINESS : ['Business']
BUSINESS : ['Business']
PRODUCTIVITY : ['Productivity']
FAMILY : ['Education']
BUSINESS : ['Business']
BUSINESS : ['Business']
FINANCE : ['Finance']
FINANCE : ['Finance']
BUSINESS : ['Business']
BUSINESS : ['Business']
BUSINESS : ['Business']
FINANCE : ['Finance']
BUSINESS : ['Business']
BUSINESS : ['Business']
BUSINESS : ['Business']
PRODUCTIVITY : ['Productivity']
TOOLS : ['Tools']
BUSINESS : ['Business']
BUSINESS : ['Business']
FAMILY : ['Entertainment']
LIFESTYLE : ['Lifestyle']
BUSINESS : ['Business']
SPORTS : ['Sports']
GAME : ['Adventure']
SOCIAL : ['Social']
PRODUCTIVITY : ['Productivity']
BUSINESS : ['Business']
BUSINESS : ['Business']
EVENTS : ['Events']
SHOPPING : ['Shopping']
BUSINESS : ['Business']
SOCIAL : ['Social']
FAMILY : ['Entertainment']
TOOLS : ['Tools']
G

TOOLS : ['Tools']
TOOLS : ['Tools']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
PRODUCTIVITY : ['Productivity']
TOOLS : ['Tools']
COMMUNICATION : ['Communication']
PRODUCTIVITY : ['Productivity']
TOOLS : ['Tools']
COMMUNICATION : ['Communication']
LIFESTYLE : ['Lifestyle']
TOOLS : ['Tools']
TOOLS : ['Tools']
FAMILY : ['Education']
FAMILY : ['Strategy']
COMMUNICATION : ['Communication']
PRODUCTIVITY : ['Productivity']
FINANCE : ['Finance']
LIFESTYLE : ['Lifestyle']
COMMUNICATION : ['Communication']
COMMUNICATION : ['Communication']
PERSONALIZATION : ['Personalization']
COMMUNICATION : ['Communication']
TOOLS : ['Tools']
LIFESTYLE : ['Lifestyle']
PERSONALIZATION : ['Personalization']
FAMILY : ['Entertainment']
COMMUNICATION : ['Communication']
FAMILY : ['Simulation']
FAMILY : ['Simulation']
PERSONALIZATION : ['Personalization']
PERSONALIZATION : ['Personalization']
GAME : ['Arcade']
TOOLS : ['Tools']
AUTO_AND_VEHICLES : ['Auto & Vehicles']
FAMILY : ['Simulation']


FINANCE : ['Finance']
BOOKS_AND_REFERENCE : ['Books & Reference']
FAMILY : ['Education']
FAMILY : ['Entertainment']
PRODUCTIVITY : ['Productivity']
VIDEO_PLAYERS : ['Video Players & Editors']
MEDICAL : ['Medical']
SOCIAL : ['Social']
SOCIAL : ['Social']
FAMILY : ['Education']
COMICS : ['Comics']
BOOKS_AND_REFERENCE : ['Books & Reference']
NEWS_AND_MAGAZINES : ['News & Magazines']
MAPS_AND_NAVIGATION : ['Maps & Navigation']
WEATHER : ['Weather']
BOOKS_AND_REFERENCE : ['Books & Reference']
FAMILY : ['Education']
BUSINESS : ['Business']
FAMILY : ['Education']
FAMILY : ['Education']
MEDICAL : ['Medical']
BOOKS_AND_REFERENCE : ['Books & Reference']
LIFESTYLE : ['Lifestyle']


The difference between the Genres and the Category columns is not very clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward as by just using the Category column it will be sufficient enough to carry out our analysis.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

# Most Popular Apps by Genre - App Store

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but this information is missing for the App Store data set. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.

In [109]:
genre_ios = freq_table(ios_free_apps, 11)
#print(genre_ios)

genre_ratings = []
for genre in genre_ios:
    #print(genre)
    total_ratings = 0 #sum of user ratings
    len_genre = 0 #number of apps specific to each genre
    for app in ios_free_apps:
        app_genre = app[11]
        if app_genre == genre:
            n_ratings = float(app[5])
            len_genre += 1
            total_ratings += n_ratings
    #print(genre, len_genre, total_ratings)
    average_ratings = round(total_ratings/len_genre)
    g_r = genre,average_ratings
    genre_ratings.append(g_r)
    #print(genre, average_ratings)
sorted_by_average_ratings = sorted(genre_ratings, key=lambda tup: tup[1], reverse = True)
for gen in  sorted_by_average_ratings:
    print(gen)

('Navigation', 86090)
('Reference', 74942)
('Social Networking', 71548)
('Music', 57327)
('Weather', 52280)
('Book', 39758)
('Food & Drink', 33334)
('Finance', 31468)
('Photo & Video', 28442)
('Travel', 28244)
('Shopping', 26920)
('Health & Fitness', 23298)
('Sports', 23009)
('Games', 22789)
('News', 21248)
('Productivity', 21028)
('Utilities', 18684)
('Lifestyle', 16486)
('Entertainment', 14030)
('Business', 7491)
('Education', 7004)
('Catalogs', 4004)
('Medical', 612)


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [110]:
for app in ios_free_apps:
    if app[11] == "Navigation":
        ratings = app[5]
        print(app[1], " ",app[5])

Waze - GPS Navigation, Maps & Real-time Traffic   345046
Google Maps - Navigation & Transit   154911
Geocaching®   12811
CoPilot GPS – Car Navigation & Offline Maps   3582
ImmobilienScout24: Real Estate Search in Germany   187
Railway Route Search   5


The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [111]:
for app in ios_free_apps:
    if app[11] == 'Reference':
        #total ratings for each app in the Reference Category
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

In [112]:
for app in ios_free_apps:
    if app[11] == "Food & Drink":
        ratings = app[5]
        print(app[1], " ",app[5])

Starbucks   303856
Domino's Pizza USA   258624
OpenTable - Restaurant Reservations   113936
Allrecipes Dinner Spinner   109349
DoorDash - Food Delivery   25947
UberEATS: Uber for Food Delivery   17865
Postmates - Food Delivery, Faster   9519
Dunkin' Donuts - Get Offers, Coupons & Rewards   9068
Chick-fil-A   5665
McDonald's   4050
Deliveroo: Restaurant Delivery - Order Food Nearby   1702
SONIC Drive-In   1645
Nowait Guest   1625
7-Eleven, Inc.   1356
Outback   805
Bon Appetit   750
Starbucks Keyboard   457
Whataburger   197
Delish Eatmoji Keyboard   154
Lieferheld - Delicious food delivery service   29
Lieferando.de   29
McDo France   22
Chefkoch - Rezepte, Kochen, Backen & Kochbuch   20
Youmiam   9
Marmiton Twist   2
Open Food Facts   1


Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. Category is mainly dominated by delivery/order/reservations services. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

In [113]:
for app in ios_free_apps:
    if app[11] == "Book":
        ratings = app[5]
        print(app[1], " ",app[5])

Kindle – Read eBooks, Magazines & Textbooks   252076
Audible – audio books, original series & podcasts   105274
Color Therapy Adult Coloring Book for Adults   84062
OverDrive – Library eBooks and Audiobooks   65450
HOOKED - Chat Stories   47829
BookShout: Read eBooks & Track Your Reading Goals   879
Dr. Seuss Treasury — 50 best kids books   451
Green Riding Hood   392
Weirdwood Manor   197
MangaZERO - comic reader   9
ikouhoushi   0
MangaTiara - love comic reader   0
謎解き   0
謎解き2016   0


# Most Popular Apps by Genre - Google Play Store

For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [114]:
display_table(android_free_apps, 5) #find out the number of installs

1,000,000+: 15.72653
100,000+: 11.55235
10,000,000+: 10.54829
10,000+: 10.19856
1,000+: 8.3935
100+: 6.91561
5,000,000+: 6.82536
500,000+: 5.56182
50,000+: 4.77211
5,000+: 4.51264
10+: 3.54242
500+: 3.2491
50,000,000+: 2.30144
100,000,000+: 2.13222
50+: 1.91787
5+: 0.78971
1+: 0.50767
500,000,000+: 0.27076
1,000,000,000+: 0.22563
0+: 0.04513
0: 0.01128


One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [115]:
android_categories = freq_table(android_free_apps,1)
#print(android_categories)

category_downloads = []
print("Category  :  Average Downloads")
#find the average number of installs for each category
#total install for all apps in the category / num of apps in each category
for category in android_categories:
    total_installs = 0 #sum of installs for each category 
    len_category = 0 #number of apps in each category
    for app in android_free_apps:
        if category == app[1]:
            len_category +=1
            n_install = app[5]
            n_install = n_install.replace(",","")
            n_install = n_install.replace("+","")
            total_installs += int(n_install)
    average_install = total_installs / len_category
    c_d = category,average_install
    category_downloads.append(c_d)
    #print(category, average_install)
#print(category_downloads)
sorted_by_average_downloads = sorted(category_downloads, key=lambda tup: tup[1], reverse = True)
for cat in sorted_by_average_downloads:
      print(cat[0].title(),": {:,.0f}".format(cat[1]))
#print(sorted_by_average_downloads)

Category  :  Average Downloads
Communication : 38,456,119
Video_Players : 24,727,872
Social : 23,253,652
Photography : 17,840,110
Productivity : 16,787,331
Game : 15,588,016
Travel_And_Local : 13,984,078
Entertainment : 11,640,706
Tools : 10,801,391
News_And_Magazines : 9,549,178
Books_And_Reference : 8,767,812
Shopping : 7,036,877
Personalization : 5,201,483
Weather : 5,074,486
Health_And_Fitness : 4,188,822
Maps_And_Navigation : 4,056,942
Family : 3,695,642
Sports : 3,638,640
Art_And_Design : 1,986,335
Food_And_Drink : 1,924,898
Education : 1,833,495
Business : 1,712,290
Lifestyle : 1,437,816
Finance : 1,387,692
House_And_Home : 1,331,541
Dating : 854,029
Comics : 817,657
Auto_And_Vehicles : 647,318
Libraries_And_Demo : 638,504
Parenting : 542,604
Beauty : 513,152
Events : 253,542
Medical : 120,551


On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [116]:
top_downloads = []
for app in android_free_apps:
    if app[1] in ["PRODUCTIVITY", "COMMUNICATION"]:
        installs = app[5]
        installs = installs.replace(",","")
        installs = installs.replace("+","")
        installs = int(installs)
        p = app[0], installs
        top_downloads.append(p)
sorted_p = sorted(top_downloads, key=lambda tup: tup[1], reverse = True)
for p in sorted_p[:10]:
      print(p[0].title(),": {:,.0f}".format(p[1]))

Whatsapp Messenger : 1,000,000,000
Messenger – Text And Video Chat For Free : 1,000,000,000
Skype - Free Im & Video Calls : 1,000,000,000
Google Chrome: Fast & Secure : 1,000,000,000
Gmail : 1,000,000,000
Hangouts : 1,000,000,000
Google Drive : 1,000,000,000
Google Duo - High Quality Video Calls : 500,000,000
Imo Free Video Calls And Chat : 500,000,000
Line: Free Calls & Messages : 500,000,000


In [117]:
#print(sorted_p[0][1])
productivity_apps_download_count = {}
for p in sorted_p:
    if p[1] not in productivity_apps_download_count:
        productivity_apps_download_count[p[1]] = 1
    else:
        productivity_apps_download_count[p[1]] += 1
#print(productivity_apps_download_count)
for key,value in productivity_apps_download_count.items():
    print("{:,.0f} downloads : {}".format(key,value))

1,000,000,000 downloads : 7
500,000,000 downloads : 9
100,000,000 downloads : 33
50,000,000 downloads : 18
10,000,000 downloads : 80
5,000,000 downloads : 43
1,000,000 downloads : 96
500,000 downloads : 22
100,000 downloads : 47
50,000 downloads : 25
10,000 downloads : 38
5,000 downloads : 28
1,000 downloads : 48
500 downloads : 21
100 downloads : 59
50 downloads : 15
10 downloads : 34
5 downloads : 4
1 downloads : 5


This shows the distribution of the downloads in the productivity category

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We have not chosen 


We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

# Remove Giant Apps (more than 50 million downloads)

The purpose of removing the giant apps like Facebook, Whatsapp, Instagram, Google Drive is to get a more accurate mean of the category downloads. These giant apps can make an impact by significantly increasing the mean and thus affecting our analysis.

In [118]:
android_other_apps = []
android_giant_apps = []

for app in android_free_apps:
    downloads = app[5]
    downloads = downloads.replace(",","")
    downloads = downloads.replace("+","")
    downloads = int(downloads)
    if(downloads < 49000000) and (downloads > 0):
        android_other_apps.append(app)
    else:
        android_giant_apps.append(app)
print(len(android_other_apps))
print(len(android_giant_apps))

8422
442


In [119]:
categories_other = []
for app in android_other_apps:
    if app[1] not in categories_other:
        categories_other.append(app[1])
  
category_downloads = []
print("Category  :  Average Downloads")
#find the average number of installs for each category
#total install for all apps in the category / num of apps in each category
for category in categories_other:
    total_installs = 0 #sum of installs for each category 
    len_category = 0 #number of apps in each category
    for app in android_other_apps:
        if category == app[1]:
            len_category +=1
            n_install = app[5]
            n_install = n_install.replace(",","")
            n_install = n_install.replace("+","")
            total_installs += int(n_install)
    average_install = total_installs / len_category
    c_d = category,average_install
    category_downloads.append(c_d)
    #print(category, average_install)
#print(category_downloads)
sorted_by_average_downloads = sorted(category_downloads, key=lambda tup: tup[1], reverse = True)
for cat in sorted_by_average_downloads:
      print(cat[0].title(),": {:,.0f}".format(cat[1]))
#print(sorted_by_average_downloads)

Category  :  Average Downloads
Entertainment : 3,808,684
Photography : 3,437,586
Game : 3,244,833
Shopping : 2,942,987
Weather : 2,392,366
Video_Players : 2,369,512
Communication : 2,319,787
Social : 2,017,797
Travel_And_Local : 1,993,455
Food_And_Drink : 1,924,898
Sports : 1,867,228
Education : 1,833,495
Health_And_Fitness : 1,827,957
Productivity : 1,735,991
Personalization : 1,705,466
Maps_And_Navigation : 1,692,173
Tools : 1,649,059
Family : 1,437,973
Books_And_Reference : 1,437,212
House_And_Home : 1,331,541
News_And_Magazines : 1,309,450
Art_And_Design : 1,149,475
Lifestyle : 1,010,129
Finance : 936,083
Dating : 854,029
Comics : 817,657
Auto_And_Vehicles : 647,318
Libraries_And_Demo : 638,504
Business : 618,802
Parenting : 542,604
Beauty : 513,152
Events : 253,542
Medical : 120,551


Category  :  Average Downloads <br>
Communication : 38,456,119 <br>
Video_Players : 24,727,872 <br>
Social : 23,253,652 <br>
Photography : 17,840,110<br>
Productivity : 16,787,331<br>
Game : 15,588,016<br>
Travel_And_Local : 13,984,078<br>
Entertainment : 11,640,706<br>
Tools : 10,801,391<br>
News_And_Magazines : 9,549,178<br>
Books_And_Reference : 8,767,812<br>
Shopping : 7,036,877<br>
Personalization : 5,201,483<br>
Weather : 5,074,486<br>
Health_And_Fitness : 4,188,822<br>
Maps_And_Navigation : 4,056,942<br>
Family : 3,695,642<br>
Sports : 3,638,640<br>
Art_And_Design : 1,986,335<br>
Food_And_Drink : 1,924,898<br>
Education : 1,833,495<br>
Business : 1,712,290<br>
Lifestyle : 1,437,816<br>
Finance : 1,387,692<br>
House_And_Home : 1,331,541<br>
Dating : 854,029<br>
Comics : 817,657<br>
Auto_And_Vehicles : 647,318<br>
Libraries_And_Demo : 638,504<br>
Parenting : 542,604<br>
Beauty : 513,152<br>
Events : 253,542<br>
Medical : 120,551<br>

# Conclusion

Compared to the output of average downloads without giant apps (above) and with giant apps (below). <br>


Also as for ios apps, we only have ratings to determine if the app is popular, we will be holding more weightage on the google play store downloads numbers. And also since the process of development would be to develop the app first on Play Store and if its successful then develop it for App Store. Thus another reason we have given more weightage to the Play Store downloads as it would give us a better sense of whether the app is successful.

As we can see, entertainment and photography are the top 2 categories with the hightest number of average downloads, so we would choose to develop an app in either of those 2 categories or create an app which incoporates both aspects.

My personal recommendation would be to create an app which has a full library of filters and users can take pictures or import them into the filter. They could import pictures of themselves or their friends and share it with them.

Both photo/video and entertainment categories also fall on the 9th and 19th position of IOS rankings.

In [120]:
for app in android_free_apps:
    if app[1] == 'ENTERTAINMENT':
        print(app[0], ':', app[5])

Complete Spanish Movies : 1,000,000+
Pluto TV - It’s Free TV : 1,000,000+
Mobile TV : 10,000,000+
TV+ : 5,000,000+
Digital TV : 5,000,000+
Motorola Spotlight Player™ : 10,000,000+
Vigo Lite : 5,000,000+
Hotstar : 100,000,000+
Peers.TV: broadcast TV channels First, Match TV, TNT ... : 5,000,000+
The green alien dance : 1,000,000+
Spectrum TV : 5,000,000+
H TV : 5,000,000+
StarTimes - Live International Champions Cup : 1,000,000+
Cinematic Cinematic : 1,000,000+
MEGOGO - Cinema and TV : 10,000,000+
Talking Angela : 100,000,000+
DStv Now : 5,000,000+
ivi - movies and TV shows in HD : 10,000,000+
Radio Javan : 1,000,000+
Talking Ginger 2 : 50,000,000+
Girly Lock Screen Wallpaper with Quotes : 5,000,000+
🔥 Football Wallpapers 4K | Full HD Backgrounds 😍 : 1,000,000+
Movies by Flixster, with Rotten Tomatoes : 10,000,000+
Low Poly – Puzzle art game : 1,000,000+
BBC Media Player : 10,000,000+
Amazon Prime Video : 50,000,000+
Adult Glitter Color by Number Book - Sandbox Pages : 1,000,000+
IMDb M

In [122]:
for app in android_free_apps:
    if app[1] == 'PHOTOGRAPHY':
        print(app[0], ':', app[5])

TouchNote: Cards & Gifts : 1,000,000+
FreePrints – Free Photos Delivered : 1,000,000+
Groovebook Photo Books & Gifts : 500,000+
Moony Lab - Print Photos, Books & Magnets ™ : 50,000+
LALALAB prints your photos, photobooks and magnets : 1,000,000+
Snapfish : 1,000,000+
Motorola Camera : 50,000,000+
HD Camera - Best Cam with filters & panorama : 5,000,000+
LightX Photo Editor & Photo Effects : 10,000,000+
Sweet Snap - live filter, Selfie photo edit : 10,000,000+
HD Camera - Quick Snap Photo & Video : 1,000,000+
B612 - Beauty & Filter Camera : 100,000,000+
Waterfall Photo Frames : 1,000,000+
Photo frame : 100,000+
Huji Cam : 5,000,000+
Unicorn Photo : 1,000,000+
HD Camera : 5,000,000+
Makeup Editor -Beauty Photo Editor & Selfie Camera : 1,000,000+
Makeup Photo Editor: Makeup Camera & Makeup Editor : 1,000,000+
Moto Photo Editor : 5,000,000+
InstaBeauty -Makeup Selfie Cam : 50,000,000+
Garden Photo Frames - Garden Photo Editor : 500,000+
Photo Frame : 10,000,000+
Selfie Camera - Photo Edito